In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from datasets import load_dataset

import time
import nltk
import numpy as np

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# english to X is only possible for T5
from datasets import load_dataset, load_from_disk

# Step 1: Load the datasets
ds_de_en = load_dataset("wmt/wmt14", "de-en")
# ds_fr_en = load_dataset("wmt/wmt15", "fr-en")
# ds_ro_en = load_dataset("wmt/wmt16", "ro-en")

# Step 2: Save the datasets to disk (replace 'path_to_save' with your desired location)
ds_de_en.save_to_disk("../wmt14_de_en")
# ds_fr_en.save_to_disk("../wmt15_fr_en")
# ds_ro_en.save_to_disk("../wmt16_ro_en")

# Step 3: Load the datasets from disk when needed
ds_de_en = load_from_disk("../wmt14_de_en")
# ds_fr_en = load_from_disk("../wmt15_fr_en")
# ds_ro_en = load_from_disk("../wmt16_ro_en")

Saving the dataset (1/1 shards): 100%|██████████| 3003/3003 [00:00<00:00, 997346.97 examples/s]
